## Measuring mapping for Multiplane Analysis

In this example we'll measure the channel to channel mapping necessary for multiplane analysis.

The mapping is a first order linear transform:

\begin{equation*}
\begin{bmatrix}
x_{f}\\
y_{f}
\end{bmatrix}
=
\begin{bmatrix}
A & B & C\\
D & E & F
\end{bmatrix}
\times
\begin{bmatrix}
1\\
x_{i}\\
y_{i}\\
\end{bmatrix}
\end{equation*}

It describes how to go between the coordinate systems of the different cameras. This relatively simple mapping is good enough for multiplane analysis, which does not require extremely high accuracy. For this analysis mapping errors on the scale of ~0.2 pixels are sufficient.

Further details about how the mapping is determined can be found in the `micrometry_mapping` notebook.


### Configuration

In real experiments you'd use a slide with sparse fiducials such as beads. The slide is quickly scanned through the focus planes of the different cameras to minimize the effects of any drift in X/Y. A single in focus image for each channel is then analyzed with `3D-DAOSTORM` or `sCMOS`.


In [ ]:
import os
os.chdir("/home/hbabcock/Data/storm_analysis/jy_testing/")
print(os.getcwd())

import numpy
numpy.random.seed(1) # Set seed so that the example is repeatable.

Generate the sample data for this example. 

In this case we're just going to simulate sparse fiducials. 

In [ ]:
import storm_analysis.jupyter_examples.multiplane_mapping as multiplane_mapping

# Make a fake sCMOS calibration file.
multiplane_mapping.makeCMOSCalibration()

# Make an sCMOS analysis XML file.
multiplane_mapping.sCMOSXML()


In [ ]:
# This is the format of the mapping file that is used for multiplane imaging.
#
# In this particular mapping each channel is translated relative to the reference channel, channel 0.
# Also there are 3 different channels.
#
mappings = {"0_0_x" : numpy.array([0.0, 1.0, 0.0]),
            "0_0_y" : numpy.array([0.0, 0.0, 1.0]),
            "0_1_x" : numpy.array([2.0, 1.0, 0.0]),
            "0_1_y" : numpy.array([5.0, 0.0, 1.0]),
            "1_0_x" : numpy.array([-2.0, 1.0, 0.0]),
            "1_0_y" : numpy.array([-5.0, 0.0, 1.0]),
            "0_2_x" : numpy.array([3.0, 1.0, 0.0]),
            "0_2_y" : numpy.array([4.0, 0.0, 1.0]),
            "2_0_x" : numpy.array([-3.0, 1.0, 0.0]),
            "2_0_y" : numpy.array([-4.0, 0.0, 1.0])}
n_planes = 3            

multiplane_mapping.makeSampleData(mappings)


### Analyze fiducial movies with sCMOS analysis

In [ ]:
import storm_analysis.sCMOS.scmos_analysis as scmosAnalysis

for i in range(n_planes):
    
    # Remove stale results, if any.
    h5_name = "c" + str(i+1) + "_map.hdf5"
    if os.path.exists(h5_name):
        os.remove(h5_name)
        
    scmosAnalysis.analyze("c" + str(i+1) + "_map.dax", h5_name, "scmos.xml")


### Check analysis

In [ ]:
import storm_analysis.jupyter_examples.overlay_image as overlay_image

for i in range(n_planes):
    
    h5_name = "c" + str(i+1) + "_map.hdf5"
    print(h5_name)
    overlay_image.overlayImage("c" + str(i+1) + "_map.dax", h5_name, 0)

### Determine mappings.

Here we determine the mapping between each channel and channel 0.

In [ ]:
import storm_analysis.micrometry.micrometry as micrometry

for i in range(1, n_planes):    
    locs1 = "c1_map.hdf5"
    locs2 = "c" + str(i+1) + "_map.hdf5"
    results = "c1_c" + str(i+1) + "_map.map"
    micrometry.runMicrometry(locs1, 
                             locs2, 
                             results, 
                             min_size = 5.0, 
                             max_size = 100.0, 
                             max_neighbors = 20, 
                             tolerance = 1.0e-2, 
                             no_plots = False)

### Check individual mappings

In [ ]:
import pickle

# Print the mappings.
for i in range(1, n_planes):
    
    results = "c1_c" + str(i+1) + "_map.map"
    with open(results, "rb") as fp:
        a_map = pickle.load(fp)

    print(results)
    for elt in sorted(a_map):
        print(elt, a_map[elt])
    print()


### Merge mappings

Note: This step is only necessary if you have more than two channels.

In [ ]:
import storm_analysis.micrometry.merge_maps as mergeMaps

# Create list of map files to merge.
to_merge = []
for i in range(1, n_planes):    
    to_merge.append("c1_c" + str(i+1) + "_map.map")
    
m_map = None
if (len(to_merge) > 1):
    # Merge.
    m_map = mergeMaps.mergeMaps(to_merge)

    for elt in sorted(m_map):
        print(elt, m_map[elt])
    print()

    # Save results for multiplane.
    with open("map.map", 'wb') as fp:
        pickle.dump(m_map, fp)
        
else:
    print("Merge not necessary")

We can also check the results against our original mapping.

In [ ]:
if m_map is not None:
    is_good = True
    for elt in sorted(m_map):
        if not numpy.allclose(m_map[elt], mappings[elt], atol = 1.0e-1, rtol = 1.0-1):
            print("map error detected for:")
            print(elt, m_map[elt], mappings[elt])
            print()
            is_good = False
    if is_good:
        print("No differences detected.")